<a href="https://colab.research.google.com/github/Vaycold/Python_DL/blob/main/Image_Transformation/%EC%82%AC%EC%A7%84%ED%99%95%EB%8C%80%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from google.colab import drive
from IPython.display import Image

warnings.filterwarnings('ignore')
#drive.mount('/content/MyDrive')